# CARGA DE DATASET Y LIBRERIAS

## Importacion de Librerias

In [49]:
import numpy as np
import pandas as pd
import pyproj
from datetime import datetime
#from openpyxl import load_workbook   

#Seteos del notebook
#pd.set_option('display.max_colwidth', 40)
#pd.options.display.min_rows
#pd.set_option('display.width', 1000)
import warnings
#warnings.filterwarnings("ignore")

## Carga del libro homicidios.xlsx 

In [26]:
ruta_excel = r'D:\PEDRO\AHORA\A LA CARGA!!\WORKING AREA\Cursos and Tuts\Henry\Labs\PIDA1\DESARROLLO DEL PI\data\homicidios.xlsx'
df_homicidios_hechos = pd.read_excel(ruta_excel, sheet_name='HECHOS')

# EXPLORACION DEL DATASET

Mostramos la estructura del libro homicidios.xlsx

In [27]:
xlsx_homicidios = pd.ExcelFile(ruta_excel)

In [28]:
nombres_hojas = xlsx_homicidios.sheet_names
for nombre in nombres_hojas:
    df = xlsx_homicidios.parse(sheet_name=nombre)
    size_bytes = df.memory_usage(deep=True).sum() 
    print(f"Hoja de trabajo: {nombre}")
    print(f"Cantidad de filas: {len(df)}")
    print(f"Cantidad de columnas: {len(df.columns)}")
    print(f"Tamaño en bytes: {round(size_bytes / 1024, 1)} Kbs")
    print("\n")

Hoja de trabajo: HECHOS
Cantidad de filas: 696
Cantidad de columnas: 21
Tamaño en bytes: 685.7 Kbs


Hoja de trabajo: DICCIONARIO_HECHOS
Cantidad de filas: 46
Cantidad de columnas: 3
Tamaño en bytes: 11.8 Kbs


Hoja de trabajo: VICTIMAS
Cantidad de filas: 717
Cantidad de columnas: 10
Tamaño en bytes: 272.0 Kbs


Hoja de trabajo: DICCIONARIO_VICTIMAS
Cantidad de filas: 21
Cantidad de columnas: 3
Tamaño en bytes: 5.3 Kbs


Hoja de trabajo: clas
Cantidad de filas: 28
Cantidad de columnas: 7
Tamaño en bytes: 10.0 Kbs




Exploracion basica

In [29]:
df_homicidios_hechos.info()
df_homicidios_hechos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


# TRATAMIENTO DE DATOS NULOS

## Verificaciones

### Identificacion de campos con valores faltantes

Computamos los campos 

In [30]:
# Calcular el conteo de valores nulos por columna
conteo_valores_nulos = df_homicidios_hechos.isnull().sum()

# Filtra las columnas con nulos mayores a  1 y ordena descendentemente
columnas_con_nulos = conteo_valores_nulos[df_homicidios_hechos.isnull().sum() > 0].sort_values(ascending = False)

print('Registros totales: ', df_homicidios_hechos.shape[0])
print("\nColumnas con valores nulos:")
print(columnas_con_nulos)

Registros totales:  696

Columnas con valores nulos:
Altura                   567
Cruce                    171
Dirección Normalizada      8
Calle                      1
dtype: int64


El DF tiene 696 registros y 4 campos con datos faltantes: 'Altura', 'Cruce', 'Dirección Normalizada' y 'Calle'

Los campos con datos faltantes parecen estar relacionados al campo 'LUGAR_DEL_HECHO'  
Para estudiarlos mejor pintamos una muestra de los campos en cuestion

In [31]:
# Muestra los nombres de los campos
print('columnas en df_homicidios_hechos: ','[' + ', '.join(df_homicidios_hechos.keys()) + ']') 

columnas en df_homicidios_hechos:  [ID, N_VICTIMAS, FECHA, AAAA, MM, DD, HORA, HH, LUGAR_DEL_HECHO, TIPO_DE_CALLE, Calle, Altura, Cruce, Dirección Normalizada, COMUNA, XY (CABA), pos x, pos y, PARTICIPANTES, VICTIMA, ACUSADO]


In [32]:
#Pinta una muestra de 25 filas entre  'LUGAR_DEL_HECHO', y 'Cruce'
df_homicidios_hechos.sample(n=10, random_state= 40)[['LUGAR_DEL_HECHO','Calle','Cruce', 'Dirección Normalizada']]
#df_homicidios_hechos.sample(n=25, random_state= 50).loc[:, 'LUGAR_DEL_HECHO':'Cruce']

,LUGAR_DEL_HECHO,Calle,Cruce,Dirección Normalizada
643,AU FRONDIZI Y SUAREZ AV.,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI y S...
132,AV MARTIN GARCIA 300,"GARCIA, MARTIN AV.",NaN,"GARCIA, MARTIN AV. 300"
508,"BRIN, MINISTRO Y SUAREZ","BRIN, MINISTRO",SUAREZ,"BRIN, MINISTRO y SUAREZ"
556,"SCALABRINI ORTIZ, RAUL AV. Y PARAGUAY","SCALABRINI ORTIZ, RAUL AV.",PARAGUAY,"SCALABRINI ORTIZ, RAUL AV. y PARAGUAY"
668,"CHILAVERT, MARTINIANO, CORONEL Y VARELA AV.","CHILAVERT, MARTINIANO, CORONEL",VARELA AV.,"CHILAVERT, MARTINIANO, CORONEL y VARELA AV."
46,AV GAONA Y ESPINOSA,GAONA AV.,ESPINOSA,GAONA AV. y ESPINOSA
683,"PAZ, GRAL. AV. Y EZEIZA","PAZ, GRAL. AV.",EZEIZA,"PAZ, GRAL. AV. y EZEIZA"
413,Santander 1974,SANTANDER,NaN,SANTANDER 1974
212,24 DE NOVIEMBRE 390,24 DE NOVIEMBRE,NaN,24 DE NOVIEMBRE 390
669,SAN LUIS 2730,SAN LUIS,NaN,SAN LUIS 2730


Comprobamos que tanto 'Calle', 'Cruce' y 'Dirección Normalizada' se crean a partir de la columna 'LUGAR_DEL_HECHO'.   

Ahora revisamos la columna 'Dirección Normalizada', que tiene 8 datos faltantes: 

In [33]:
normalizados_faltantes = df_homicidios_hechos[df_homicidios_hechos['Dirección Normalizada'].isnull()][['LUGAR_DEL_HECHO','Calle','Dirección Normalizada']]
normalizados_faltantes

,LUGAR_DEL_HECHO,Calle,Dirección Normalizada
38,AUTOPISTA LUGONES PK 10000,"LUGONES, LEOPOLDO AV.",NaN
106,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA BUENOS AIRES - LA PLATA,NaN
119,SD,NaN,NaN
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA PERITO MORENO,NaN
181,AU DELLEPIANE 2400,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NaN
313,AUTOPISTA LUGONES KM 4.7,"LUGONES, LEOPOLDO AV.",NaN
546,"LUGONES, LEOPOLDO AV. KM 6,1","LUGONES, LEOPOLDO AV.",NaN
621,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA BUENOS AIRES - LA PLATA,NaN


Se infiere que los datos faltantes en 'Dirección Normalizada' no se normalizaron por que los siniestros ocurrieron en autopistas   Asimismo se muestra el dato faltante en 'Calle' de indice 119 coincidiendo con uno de 'Dirección Normalizada'  

In [34]:
# Captura la fila con indice 119
registro_indice_119 = df_homicidios_hechos.iloc[119]

# Transpone para mostrarlo en horizontal
registro_indice_119.to_frame().T 

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD


El registro 119 no tiene datos significativos. 

## Conclusiones y Medidas Tomadas

* Los datos faltantes en 'Altura' representan una enorme proporción del conjunto de datos totales: SE ELIMINA
* El campo 'Cruce' tiene 171 valores faltantes y poco peso significativo por si mismo: SE ELIMINA
* Los datos de 'Dirección Normalizada' son muy similares a los del campo 'Calle': SE IMPUTAN LOS 8 DATOS FALTANTES POR EL CAMPO 'Calle'
* El unico dato faltante de 'Calle' se encuentra en un registro sin datos significativos: SE ELIMINA Y SE RESETEA EL INDICE

Eliminacion de 'Altura' y 'Cruce'

In [35]:
df_homicidios_hechos = df_homicidios_hechos.drop(columns=['Altura'])
df_homicidios_hechos = df_homicidios_hechos.drop(columns=['Cruce'])

Imputacion de 'Dirección Normalizada'

In [36]:
# Reemplaza todos los valores nulos en 'Dirección Normalizada' con los valores de 'Calle'
df_homicidios_hechos['Dirección Normalizada'].fillna(df_homicidios_hechos['Calle'], inplace=True)
print("Se imputaron todos los datos los datos faltantes en 'Dirección Normalizada'")
#Checkamos los cambios
df_homicidios_hechos[df_homicidios_hechos['Dirección Normalizada'] == df_homicidios_hechos['Calle']][['LUGAR_DEL_HECHO','Calle','Dirección Normalizada']]

Se imputaron todos los datos los datos faltantes en 'Dirección Normalizada'


,LUGAR_DEL_HECHO,Calle,Dirección Normalizada
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI
38,AUTOPISTA LUGONES PK 10000,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV."
71,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
106,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA BUENOS AIRES - LA PLATA,AUTOPISTA BUENOS AIRES - LA PLATA
139,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO
176,AV. LEOPOLDO LUGONES PKM 6900,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV."
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA PERITO MORENO,AUTOPISTA PERITO MORENO
181,AU DELLEPIANE 2400,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
256,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI
313,AUTOPISTA LUGONES KM 4.7,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV."


Eliminacion del indice 119 y reseteo del index

In [37]:
df_homicidios_hechos = df_homicidios_hechos.drop(119)
print("Datos faltantes en 'Calle':")
print(df_homicidios_hechos[['Calle']].isnull().sum())

df_homicidios_hechos.reset_index(drop=True, inplace=True)

Datos faltantes en 'Calle':
Calle    0
dtype: int64


Confirmamos que no haya datos faltantes en df_homicidios_hechos

In [38]:
print('Datos faltantes:')
print(df_homicidios_hechos.isnull().sum())

Datos faltantes:
ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
Calle                    0
Dirección Normalizada    0
COMUNA                   0
XY (CABA)                0
pos x                    0
pos y                    0
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
dtype: int64


---
<br/><br/>

# TRATAMIENTO DE VALORES INVALIDOS

## Verificacion del campo 'Numero de Victimas'  

In [39]:
# Crea un nuevo DataFrame con las filas que cumplen con la condición
df_valores_diferentes = df_homicidios_hechos[~df_homicidios_hechos['N_VICTIMAS'].isin([1, 2, 3])]

# Muestra el nuevo DataFrame
df_valores_diferentes

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


In [40]:
# Información sobre el DataFrame, incluyendo tipos de datos y valores faltantes
df_homicidios_hechos.info()
df_homicidios_hechos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  Calle                  695 non-null    object        
 11  Dirección Normalizada  695 non-null    object        
 12  COMUNA                 695 non-null    int64         
 13  XY (C

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


##  Verificacion del campo 'HORA'

In [41]:
# Define un patrón de expresión regular para el formato deseado (HH:MM:SS)
patron_formato = r'^\d{2}:\d{2}:\d{2}'

# Utiliza str.match para encontrar las filas que coinciden con el patrón
valores_no_validos = df_homicidios_hechos[~df_homicidios_hechos['HORA'].astype(str).str.match(patron_formato)]

# Muestra las filas que no tienen el formato deseado
valores_no_validos

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
438,2019-0022,1,2019-03-15,2019,3,15,1900-01-02 00:00:00,0,ENTRE RIOS AV. BAJO PUENTE AUTOPISTA (1260 APROX),AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 1260,1,Point (106608.39392805 100635.27735528),-58.39123782,-34.62352245,AUTO-SD,AUTO,SD
440,2019-0024,1,2019-03-19,2019,3,19,1900-01-14 07:12:00,7,ANTARTIDA ARGENTINA AV. 1325,AVENIDA,ANTARTIDA ARGENTINA AV.,ANTARTIDA ARGENTINA AV. 1325,1,Point (108368.93688513 104860.74716674),-58.37208118,-34.58541957,MOTO-CARGAS,MOTO,CARGAS
495,2019-0080,1,2019-09-17,2019,9,17,1900-01-04 18:40:00,18,"CABILDO AV. y PAZ, GRAL. AV.",GRAL PAZ,CABILDO AV.,"CABILDO AV. y PAZ, GRAL. AV.",13,Point (98817.74986778 110096.31367495),-58.47617802,-34.53825652,PEATON-MOTO,PEATON,MOTO
517,2019-0103,1,2019-12-18,2019,12,18,SD,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Tenemos 4 valores invalidos en 'HORA'. Los imputamos con el valor mas frecuente de 'HORA' y modificamos la columna 'HH' con el valor pertinente

In [42]:
indices_a_modificar = [438, 440, 495, 517]

# Encuentra el valor más repetido en 'Hora Exacta'
hora_mas_frecuente = df_homicidios_hechos['HORA'].mode()[0]
print(f'Valor más frecuente en "HORA": {hora_mas_frecuente}')

for indice in indices_a_modificar:    
    # Modifica los registros seleccionados
    print(f"Valor del indice {indice} antes de la actualización:", df_homicidios_hechos.at[indice, 'HORA'])
    df_homicidios_hechos.at[indice, 'HORA'] = hora_mas_frecuente
    print(f"Valor del indice {indice} después de la actualización:", df_homicidios_hechos.at[indice, 'HORA'])

    # Actualizar la columna 'HH' en los índices especificados
    hora_mas_frecuente_string = df_homicidios_hechos['HORA'].mode()[0].strftime('%H:%M:%S')
    df_homicidios_hechos.at[indice, 'HH'] = int(hora_mas_frecuente_string.split(':')[0])


# Verificar que los cambios se hayan aplicado correctamente
df_homicidios_hechos.loc[indices_a_modificar]

Valor más frecuente en "HORA": 14:00:00
Valor del indice 438 antes de la actualización: 1900-01-02 00:00:00
Valor del indice 438 después de la actualización: 14:00:00
Valor del indice 440 antes de la actualización: 1900-01-14 07:12:00
Valor del indice 440 después de la actualización: 14:00:00
Valor del indice 495 antes de la actualización: 1900-01-04 18:40:00
Valor del indice 495 después de la actualización: 14:00:00
Valor del indice 517 antes de la actualización: SD
Valor del indice 517 después de la actualización: 14:00:00


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
438,2019-0022,1,2019-03-15,2019,3,15,14:00:00,14,ENTRE RIOS AV. BAJO PUENTE AUTOPISTA (1260 APROX),AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 1260,1,Point (106608.39392805 100635.27735528),-58.39123782,-34.62352245,AUTO-SD,AUTO,SD
440,2019-0024,1,2019-03-19,2019,3,19,14:00:00,14,ANTARTIDA ARGENTINA AV. 1325,AVENIDA,ANTARTIDA ARGENTINA AV.,ANTARTIDA ARGENTINA AV. 1325,1,Point (108368.93688513 104860.74716674),-58.37208118,-34.58541957,MOTO-CARGAS,MOTO,CARGAS
495,2019-0080,1,2019-09-17,2019,9,17,14:00:00,14,"CABILDO AV. y PAZ, GRAL. AV.",GRAL PAZ,CABILDO AV.,"CABILDO AV. y PAZ, GRAL. AV.",13,Point (98817.74986778 110096.31367495),-58.47617802,-34.53825652,PEATON-MOTO,PEATON,MOTO
517,2019-0103,1,2019-12-18,2019,12,18,14:00:00,14,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


---
<br/><br/>

## Verificacion del campo 'PARTICIPANTES' y sus campos dependientes
'PARTICIPANTES' tiene varios valores invalidos tipo 'MOTO-SD', 'SD-AUTO', 'SD-SD'.  
LOS CAMPOS 'VICTIMA' y 'ACUSADO' herendan parcialmente estos valores invalidos. Lo tratamos de la siguiente manera:
* Imputamos los valores que contienen 'SD' del campo 'VICTIMA' con su valor mas frecuente. 
* Imputamos los valores que contienen 'SD' del campo 'ACUSADO' con su valor mas frecuente.
* Imputamos los valotres que contienen 'SD' del campo 'PARTICIPANTES'con los valores de 'VICTIMA' y 'ACUSADO'

Checkamos si hay variantes en minuscula de 'SD' y mostramos los filas con valores invalidos

In [43]:
# Filtrar las filas que contienen las cadenas mencionadas en las columnas 'VICTIMA' y 'ACUSADO'
# El argumento case=False considerará automáticamente todas las combinaciones de mayúsculas y minúsculas: 'sd', 'SD', 'sD', 'Sd'
filtro = df_homicidios_hechos['VICTIMA'].str.contains('sd', case=False) | df_homicidios_hechos['ACUSADO'].str.contains('sd', case=False)

# Aplicar el filtro al DataFrame
df_valores_invalidos = df_homicidios_hechos[filtro][['PARTICIPANTES','VICTIMA', 'ACUSADO']]

# Captura los indices de las filas con valores invalidosa
indices_valores_invalidos = df_valores_invalidos.index.tolist()
print(len(df_valores_invalidos))

df_valores_invalidos


26


,PARTICIPANTES,VICTIMA,ACUSADO
3,MOTO-SD,MOTO,SD
32,MOTO-SD,MOTO,SD
35,SD-SD,SD,SD
38,MOTO-SD,MOTO,SD
57,AUTO-SD,AUTO,SD
76,SD-SD,SD,SD
83,PEATON-SD,PEATON,SD
88,SD-CARGAS,SD,CARGAS
92,SD-SD,SD,SD
105,PEATON-SD,PEATON,SD


No vemos variantes en minusculas de 'SD', ahora imputamos

In [44]:
# Reemplazar 'SD' en 'VICTIMA' con el valor más frecuente en 'VICTIMA'
#valor_mas_frecuente_victima = df_homicidios_hechos['VICTIMA'].mode().iloc[0]
#df_homicidios_hechos['VICTIMA'] = df_homicidios_hechos['VICTIMA'].replace('SD', valor_mas_frecuente_victima)

# Reemplazar 'SD' en 'ACUSADO' con el valor más frecuente en 'ACUSADO'
#valor_mas_frecuente_acusado = df_homicidios_hechos['ACUSADO'].mode().iloc[0]
#df_homicidios_hechos['ACUSADO'] = df_homicidios_hechos['ACUSADO'].replace('SD', valor_mas_frecuente_acusado)

# Concatenar 'VICTIMA' y 'ACUSADO' y asignar el resultado a 'PARTICIPANTES'
#df_homicidios_hechos['PARTICIPANTES'] = df_homicidios_hechos['VICTIMA'] + '-' + df_homicidios_hechos['ACUSADO']

# Captura el valor más frecuente de 'VICTIMA' y 'ACUSADO'
valor_mas_frecuente_victima = df_homicidios_hechos['VICTIMA'].mode().iloc[0]
valor_mas_frecuente_acusado = df_homicidios_hechos['ACUSADO'].mode().iloc[0]

for indice in indices_valores_invalidos:

    # Imputa cada valor no valido en 'VICTIMA' y 'ACUSADO' con el valor masfrecuente
    df_homicidios_hechos.at[indice, 'VICTIMA'] = valor_mas_frecuente_victima
    df_homicidios_hechos.at[indice, 'ACUSADO'] = valor_mas_frecuente_acusado

    # Imputa cada valor no valido en 'PARTICIPANTES'
    valor_victima = df_homicidios_hechos.at[indice, 'VICTIMA']
    valor_acusado = df_homicidios_hechos.at[indice, 'ACUSADO']
    valor_concatenado = f"{valor_victima}-{valor_acusado}"
    df_homicidios_hechos.at[indice, 'PARTICIPANTES'] = valor_concatenado

df_homicidios_hechos.loc[indices_valores_invalidos][['PARTICIPANTES','VICTIMA','ACUSADO']]

,PARTICIPANTES,VICTIMA,ACUSADO
3,MOTO-AUTO,MOTO,AUTO
32,MOTO-AUTO,MOTO,AUTO
35,MOTO-AUTO,MOTO,AUTO
38,MOTO-AUTO,MOTO,AUTO
57,MOTO-AUTO,MOTO,AUTO
76,MOTO-AUTO,MOTO,AUTO
83,MOTO-AUTO,MOTO,AUTO
88,MOTO-AUTO,MOTO,AUTO
92,MOTO-AUTO,MOTO,AUTO
105,MOTO-AUTO,MOTO,AUTO


In [45]:
# Información sobre el DataFrame, incluyendo tipos de datos y valores faltantes
df_homicidios_hechos.info()
df_homicidios_hechos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  Calle                  695 non-null    object        
 11  Dirección Normalizada  695 non-null    object        
 12  COMUNA                 695 non-null    int64         
 13  XY (C

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


# MANEJO DE VALORES ATIPICOS

## Coordenadas Geograficas 'Longitud Geografica' y 'Latitud Geografica'
Reemplazamos los valores con un  '.' a nan 

In [46]:

# Reemplaza los valores que contienen '.' con NaN en 'pos x' y 'pos y'
df_homicidios_hechos['pos x'].replace('.', np.nan, inplace=True)
df_homicidios_hechos['pos y'].replace('.', np.nan, inplace=True)

# Muestra los valores nan en 'pos x','pos y'
coordenadas_faltantes = df_homicidios_hechos[df_homicidios_hechos['pos x'].isnull()][['ID','pos x','pos y']]
#print(df[['pos x','pos y']].dtypes)
coordenadas_faltantes



,ID,pos x,pos y
38,2016-0052,NaN,NaN
106,2016-0136,NaN,NaN
138,2016-0174,NaN,NaN
175,2017-0042,NaN,NaN
179,2017-0050,NaN,NaN
180,2017-0051,NaN,NaN
255,2017-0140,NaN,NaN
312,2018-0039,NaN,NaN
545,2020-0026,NaN,NaN
558,2020-0039,NaN,NaN


# TRATAMIENTOS FINALES

## Exploracion Basica

In [47]:
# Información sobre el DataFrame, incluyendo tipos de datos y valores faltantes
df_homicidios_hechos.info()
df_homicidios_hechos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  Calle                  695 non-null    object        
 11  Dirección Normalizada  695 non-null    object        
 12  COMUNA                 695 non-null    int64         
 13  XY (C

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


## Redefinicion de los Campos

* El campo 'Dia' provee poca o nula significacion para el posterior EDA. Una aproximacion interesante seria conocer el dia de la semana en que ocurrio el siniestro
* Los nombres de los campos no so claros, lo que no ayudan a su comprension

### Acciones a Realizar:

* Generar el campo 'Dia de Semana' a partir del campo 'Fecha de Siniestro'
* Cambiar el nombre de las columnas 

Se genera el campo 'Dia de Semana'

In [51]:
# Crea la función para obtener el día de la semana a partir del campo 'Fecha de Siniestro'
def obtener_dia_semana(fecha_str):
    fecha = datetime.strptime(fecha_str, '%Y-%m-%d')
    return fecha.strftime('%A')  # Devuelve el nombre del día de la semana

# Aplica la función y crea la una nueva columna
#df_homicidios_hechos['Dia de Semana'] = df_homicidios_hechos['FECHA'].apply(obtener_dia_semana) 
df_homicidios_hechos['Dia de Semana'] = df_homicidios_hechos['FECHA'].dt.day_name()

# Cambia los nombres de los días a español
mapeo_dias = {'Monday': 'Lunes','Tuesday': 'Martes','Wednesday': 'Miércoles','Thursday': 'Jueves',
                'Friday': 'Viernes', 'Saturday': 'Sábado', 'Sunday': 'Domingo'}
df_homicidios_hechos['Dia de Semana'] = df_homicidios_hechos['Dia de Semana'].replace(mapeo_dias)

# Mueve la columna 'Dia de la Semana' de la última posición a la quinta
columnas = list(df_homicidios_hechos.columns)
nueva_posicion = 5
columnas.insert(nueva_posicion, columnas.pop()) 
df_homicidios_hechos = df_homicidios_hechos[columnas] #reordena el DataFrame con el nuevo orden

# Mostrar el DataFrame con los nombres de los días en español
df_homicidios_hechos.head(1)

,ID,N_VICTIMAS,FECHA,AAAA,MM,Dia de Semana,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,Viernes,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


In [50]:
print(df_homicidios_hechos['FECHA'].dtype)


datetime64[ns]


Se cambia el nombre de los campos

In [52]:
# Setea los nombres de campos a cambiar
campos_a_cambiar = {'N_VICTIMAS': 'Nro de Victimas', 'FECHA': 'Fecha de Siniestro', 'AAAA': 'Año', 'MM': 'Mes', 'DD': 'Dia', 'HORA': 'Hora Exacta', 'HH': 'Hora del Dia', 'LUGAR_DEL_HECHO': 'Direccion del Siniestro', 'TIPO_DE_CALLE': 'Tipo de Calle', 'COMUNA': 'Comuna', 'XY (CABA)': 'Geocodigo Plano del Siniestro', 'pos x': 'Longitud Geografica', 'pos y': 'Latitud Geografica', 'PARTICIPANTES': 'Vehiculos Prticipantes', 'VICTIMA': 'Vehiculo de la Victima', 'ACUSADO': 'Vehiculo del Acusado'}

# Cambia los nombres de las columnas'
df_homicidios_hechos.rename(columns = campos_a_cambiar, inplace=True)

# Muestra una lista des cambios
print('Se han modificado campos: ')
print(pd.Series(campos_a_cambiar).T)

# Mostrar el DataFrame con los nombres de columnas cambiados
df_homicidios_hechos.head(1)

Se han modificado campos: 
N_VICTIMAS                       Nro de Victimas
FECHA                         Fecha de Siniestro
AAAA                                         Año
MM                                           Mes
DD                                           Dia
HORA                                 Hora Exacta
HH                                  Hora del Dia
LUGAR_DEL_HECHO          Direccion del Siniestro
TIPO_DE_CALLE                      Tipo de Calle
COMUNA                                    Comuna
XY (CABA)          Geocodigo Plano del Siniestro
pos x                        Longitud Geografica
pos y                         Latitud Geografica
PARTICIPANTES             Vehiculos Prticipantes
VICTIMA                   Vehiculo de la Victima
ACUSADO                     Vehiculo del Acusado
dtype: object


,ID,Nro de Victimas,Fecha de Siniestro,Año,Mes,Dia de Semana,Dia,Hora Exacta,Hora del Dia,Direccion del Siniestro,Tipo de Calle,Calle,Dirección Normalizada,Comuna,Geocodigo Plano del Siniestro,Longitud Geografica,Latitud Geografica,Vehiculos Prticipantes,Vehiculo de la Victima,Vehiculo del Acusado
0,2016-0001,1,2016-01-01,2016,1,Viernes,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


## Revisiones Finales

In [53]:
print(df_homicidios_hechos.info())
df_homicidios_hechos.head(4)[['ID','Geocodigo Plano del Siniestro','Longitud Geografica','Latitud Geografica','Vehiculos Prticipantes','Vehiculo de la Victima','Vehiculo del Acusado']]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   ID                             695 non-null    object        
 1   Nro de Victimas                695 non-null    int64         
 2   Fecha de Siniestro             695 non-null    datetime64[ns]
 3   Año                            695 non-null    int64         
 4   Mes                            695 non-null    int64         
 5   Dia de Semana                  695 non-null    object        
 6   Dia                            695 non-null    int64         
 7   Hora Exacta                    695 non-null    object        
 8   Hora del Dia                   695 non-null    object        
 9   Direccion del Siniestro        695 non-null    object        
 10  Tipo de Calle                  695 non-null    object        
 11  Calle              

,ID,Geocodigo Plano del Siniestro,Longitud Geografica,Latitud Geografica,Vehiculos Prticipantes,Vehiculo de la Victima,Vehiculo del Acusado
0,2016-0001,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-AUTO,MOTO,AUTO


In [ ]:
df_homicidios_hechos.sample(n=15, random_state= 50)

,ID,Nro de Victimas,Fecha de Siniestro,Año,Mes,Dia de Semana,Dia,Hora Exacta,Hora del Dia,Direccion del Siniestro,Tipo de Calle,Calle,Dirección Normalizada,Comuna,Geocodigo Plano del Siniestro,Longitud Geografica,Latitud Geografica,Vehiculos Prticipantes,Vehiculo de la Victima,Vehiculo del Acusado
16,2016-0022,1,2016-02-21,2016,2,Domingo,21,06:00:00,6,AV 9 DE JULIO Y ADOLFO ALSINA,AVENIDA,9 DE JULIO AV.,"9 DE JULIO AV. y ALSINA, ADOLFO",1,Point (107507.39502374 102046.08333077),-58.381447,-34.610799,MOTO-AUTO,MOTO,AUTO
195,2017-0068,1,2017-05-27,2017,5,Sábado,27,19:50:00,19,AV. TTE. FRANCISCO RABANAL 2950,AVENIDA,"RABANAL, FRANCISCO, INTENDENTE AV.","RABANAL, FRANCISCO, INTENDENTE AV. 2950",8,Point (102520.03961549 96105.29592085),-58.435807,-34.664375,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
77,2016-0102,1,2016-08-10,2016,8,Miércoles,10,23:37:00,23,AV EVA PERON Y JOSE LEON SUAREZ,AVENIDA,"PERON, EVA AV.","PERON, EVA AV. y SUAREZ, JOSE LEON",9,Point (96625.80612216 95129.90776954),-58.500116,-34.673165,MOTO-CARGAS,MOTO,CARGAS
655,2021-0058,1,2021-07-21,2021,7,Miércoles,21,00:40:00,0,AV. GRAL. PAZ Y VEDIA,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y VEDIA",12,Point (97603.26759351 109241.33811464),-58.489410,-34.545962,PEATON-AUTO,PEATON,AUTO
208,2017-0084,1,2017-07-05,2017,7,Miércoles,5,12:49:00,12,AV. DEL LIBERTADOR Y RODRIGUEZ PEÑA,AVENIDA,DEL LIBERTADOR AV.,DEL LIBERTADOR AV. y RODRIGUEZ PEÃ‘A,2,Point (107180.90591568 104682.54945160),-58.385029,-34.587035,PEATON-PASAJEROS,PEATON,PASAJEROS
18,2016-0026,1,2016-02-28,2016,2,Domingo,28,07:30:00,7,HOLMBERG Y VEDIA,CALLE,HOLMBERG,HOLMBERG y VEDIA,12,Point (97350.75566590 109083.19420265),-58.492162,-34.547386,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
522,2020-0003,1,2020-01-11,2020,1,Sábado,11,06:00:00,6,RIVADAVIA AV. Y COMBATE DE LOS POZOS,AVENIDA,RIVADAVIA AV.,RIVADAVIA AV. y COMBATE DE LOS POZOS,3,Point (106413.35913221 102215.84332055),-58.393376,-34.609276,PEATON-AUTO,PEATON,AUTO
2,2016-0003,1,2016-01-03,2016,1,Domingo,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO
295,2018-0022,1,2018-02-24,2018,2,Sábado,24,00:00:00,0,"Montes De Oca, Manuel y Olavarria",AVENIDA,"MONTES DE OCA, MANUEL AV.","MONTES DE OCA, MANUEL AV. y OLAVARRIA",4,Point (108149.86840397 98842.98534187),-58.374412,-34.639668,MOTO-CARGAS,MOTO,CARGAS
680,2021-0083,1,2021-11-09,2021,11,Martes,9,00:35:00,0,CABILDO AV. Y OLAZABAL,AVENIDA,CABILDO AV.,CABILDO AV. y OLAZABAL,13,Point (100459.06991201 107666.84775309),-58.458297,-34.560158,MOTO-AUTO,MOTO,AUTO


# GUARDAMOS EL TRABAJO

In [55]:
df_homicidios_hechos.to_csv('../data/df_homicidios_hechos_prepared.csv', index=False)
